# Implementação K-NN

Referencias:


In [1]:
import pandas as pd
import numpy as np
from random import randint
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import warnings
warnings.filterwarnings('ignore')

dataset = pd.read_csv('dataset/iris.csv')

dataset.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### Funções auxiliares
- Euclidian distance

In [2]:
def euclidian_distance(tuple_one, tuple_two):
    ed = 0
    for t1, t2 in zip(tuple_one, tuple_two):
        ed += (t1-t2) ** 2
    return np.sqrt(ed)

### Variaveis auxiliares

In [24]:
last_col = dataset.columns[len(dataset.columns)-1]
k = 3
classes = list(dataset[last_col].unique())

### Implementação do Naive Bayes

Para a criação dos dados, interajo 30 vezes, e para cada interação, utilizando uma variavel de controle ``i`` da primeira interação, eu utilizo um calculo matemático (``(80*i/30+20*(30-i)/30)/100``) que varia entre 20% até 80%.
Assim eu consigo montar 2 datasets, um de treino e outro para teste.
Para cada interação, ele vai reordenando o dataset de maneira aleatória (pandas já faz isso automaticamente) para treinar e para testar.

#### Treinar e Testar
O algoritmo para implementar o Naive Bayes, foi utilizando interando todos o dataset de ``teste`` e fazendo a seguinte formula ``P(atributo_1 | classe) x P(atributo_2 | classe) ... P(atributo_n | classe) x P(classe)`` e para todas as classes utilizando o **Maximum Likelihood** para buscar a classe com maior probabilidade e predizer o resultado.
Com o resultado desse dado eu faço a verificação com cada dataset de teste da interação

In [26]:
def knn (dataset,value, kn):
    tuple_d = [] #Tupla formato (row_index, distancia do valor)
    for i,row in enumerate(dataset.values):
        tuple_d.append((i, euclidian_distance(value,tuple(row))))
    tuple_d = sorted(tuple_d,key= lambda x: x[1])
    
    minor = [tuple_d.pop(0) for i in range(0,kn)] # Array com os K menores valores da tupla

    classes.sort()
    print(minor)
    counts_classes_dist = [0 for r in range(0,len(classes))]
    for point in minor:
        index_result = classes.index(dataset.loc[point[0]][last_col])
        counts_classes_dist[index_result] += 1
    
    classes_points = list(enumerate(counts_classes_dist))
    result = max(classes_points,key=lambda x: x[1])
    return classes[result[0]]
    
    
    
def train(dataset_train, dataset_test):
    count_correct = 0
    count_incorrect = 0
    
    count_by_classes_correct = [0 for i in range(0,len(classes))]
    count_by_classes_incorrect = [0 for i in range(0,len(classes))]
    
    for index, row in dataset_test.iterrows():
        
        tuple_t = list(row)
        tuple_t.pop()
        result = knn(dataset_train,tuple(tuple_t),k)
        

        #Result
        if result == row[last_col]:
            count_correct += 1
            count_by_classes_correct[classes.index(result)] += 1
        else:
            count_incorrect += 1
            count_by_classes_incorrect[classes.index(result)] += 1
        
    return (count_correct, count_incorrect, count_by_classes_correct, count_by_classes_incorrect)

In [27]:
def run_nth(percentage, number):
    percentages_correct = list()
    
    
    for i in range(0,number):

        # Essa função pega o dataset e separa uma fração dele, e reordena
        ds_train=dataset.sample(frac=percentage, random_state=i*randint(0,100)).reset_index()
        ds_test=dataset.drop(ds_train.index) # Pega o que sobrou do dataset de treino

        correct, incorrect, correct_zero, correct_one, incorrect_zero, incorrect_one = naive_bayes(ds_train,ds_test)
        
        if (correct_zero+incorrect_zero) != 0:
            prob_correct_zero.append(correct_zero/(correct_zero+incorrect_zero))
        
        if (correct_one+incorrect_one):
            prob_correct_one.append(correct_one/(correct_one+incorrect_one))
        
        percentages_correct.append(correct/(correct+incorrect))
    
        
    return (percentages_correct, prob_correct_zero, prob_correct_one)

### Trata o resultado do treino e teste
Faz o calculo da porcentagem de acordo com a quantidade de corretas e incorretas

In [ ]:
percents, prob_zero, prob_one = run_nth(0.5,10)

### Taxa de Acerto

In [ ]:
taxa_acerto_min=np.min(percents)
taxa_acerto_max=np.max(percents)
taxa_acerto_med=np.mean(percents)

print('Taxa de Acerto')
print('------------------')
print('Minimo: ' + str(taxa_acerto_min))
print('Máxima: ' + str(taxa_acerto_max))
print('Média: '+str(taxa_acerto_med))

### Taxa de Acerto Médio por Classe

In [ ]:
taxa_acerto_zero = np.mean(prob_zero)
taxa_acerto_one = np.mean(prob_one)

print('Taxa de Acerto Médio por classe')
print('------------------')
print('Classe 0: ' + str(taxa_acerto_zero))
print('Classe 1: ' + str(taxa_acerto_one))

### Gráfico de acerto
Gráfico contendo a ``Probabilidade de acerto X Porcentagem do dataset de treino``.


In [ ]:
fig = plt.figure(1, (7,4))
ax = fig.add_subplot(1,1,1)

ax.plot(percents)
ax.set_title('Gráfico de acerto')

percentFormat = mtick.PercentFormatter(1)

ax.xaxis.set_major_formatter(percentFormat)
ax.yaxis.set_major_formatter(percentFormat)
ax.set_ylabel('Probabilidade de acerto')

ax.set_xlabel('Porcentagem do dataset de treino')

plt.grid(True)
plt.show()

Para cada porcentagem do dataset de treino, o restante dos dados foram utilizado para teste.
Por exemplo, caso eu tenhoa usado 20% para treino, 80% foram utilizado para teste (Não está explicito no gráfico)

In [7]:
list(dataset[last_col].unique())

['setosa', 'versicolor', 'virginica']

In [18]:
test =  list(dataset.iloc[1])

In [21]:
test.pop()

(4.9, 3.0, 1.4, 0.2)

In [5]:
list(enumerate([1,10,10,2]))

[(0, 1), (1, 10), (2, 10), (3, 2)]

In [6]:
max(list(enumerate([1,10,10,2])), key=lambda x: x[1])

(1, 10)